Testing theory power spectra generator script

## Test theory

In [1]:
#auto load changes 
%load_ext autoreload
%autoreload 2

%load_ext line_profiler

In [2]:
import sys
import pickle
import camb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

sys.path.insert(0,'/global/homes/t/tanveerk/SkyLens/') #path to skylens
sys.path.insert(0,'/global/homes/t/tanveerk/SkyLens/skylens') #path to skylens
sys.path.insert(0,'/global/homes/t/tanveerk/lselgsXplanck/src/') #path to helper functions

import healpy as hp
import skylens
import utilsCross #helper functions


from time import time

In [4]:
#setting up virtual dask cluster for calculations. Adjust memory and threads according to your system.
from distributed import LocalCluster
from dask.distributed import Client 
#http://distributed.readthedocs.io/en/latest/_modules/distributed/worker.html
c=LocalCluster(n_workers=1,processes=False,memory_limit='150gb',threads_per_worker=4,
               memory_spill_fraction=.99,memory_monitor_interval='2000ms')
client=Client(c)

/global/homes/t/tanveerk/.conda/envs/cp_env/lib/python3.7/site-packages/distributed/node.py:161: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 46067 instead
  f"Port {expected} is already in use.\n"


In [5]:
#Import wigner files

wigner_files={} #run Gen_wig_m0.py to generate these files and then pass their path here. These are large files, make sure they are not included in git.
# In file names l3500 refers to ell_max for which wigner function was calculated. It should be same or larger than ell_max in your measurements. 
# |w2100 is the max ell for window. This should be atleast 2 X ell_max in the measurements.
wigner_files[0]= '/global/cscratch1/sd/tanveerk/wig3j_l3072_w6144_0_reorder.zarr/'

In [6]:
from astropy.cosmology import Planck18_arXiv_v2 as cosmo_planck

cosmo_fid=dict({'h':cosmo_planck.h,
                'Omb':cosmo_planck.Ob0,
                'Omd':cosmo_planck.Om0-cosmo_planck.Ob0,
                's8':0.811,
                'Om':cosmo_planck.Om0,
                'Ase9':2.2,
                'mnu':cosmo_planck.m_nu[-1].value,
                'Omk':cosmo_planck.Ok0,
                'tau':0.06,
                'ns':0.965,
                'OmR':cosmo_planck.Ogamma0+cosmo_planck.Onu0,
                'w':-1,
                'wa':0,
                'T_cmb':cosmo_planck.Tcmb0, 
                'Neff':cosmo_planck.Neff,
                'z_max':1090,
                'use_astropy':True})

In [7]:
cosmo_params = cosmo_fid

In [8]:
test = False

if test:
    NSIDE = 256
    lmax_cl = 3 * NSIDE
    binsize = 10
else:
    NSIDE = 1024
    lmax_cl = 3 * NSIDE - 1
    binsize = 50

In [9]:
#setup parameters
lmin_cl=0
l0=np.arange(lmin_cl,lmax_cl)

#following defines the ell bins. Using log bins in example, feel free to change.
lmin_cl_Bins=50
lmax_cl_Bins=lmax_cl-10
Nl_bins=20
#l_bins=np.int64(np.logspace(np.log10(lmin_cl_Bins),np.log10(lmax_cl_Bins),Nl_bins))
l_bins = np.arange(lmin_cl_Bins, lmax_cl_Bins, binsize)
#lb=np.sqrt(l_bins[1:]*l_bins[:-1])
lb=0.5*(l_bins[1:]+l_bins[:-1])

l=l0

do_cov=False # if you want to get covariance. Covariance is slow and this should be false if you are calling skylens inside mcmc.
bin_cl=True #bin the theory and covaraince. 

use_window=False #if you want to include the window effect. Code will return pseudo-cl and pseudo-cl covariance
store_win=True # to store window, for some internal reasons. leave it to true.
window_lmax=3*NSIDE - 1 #smaller value for testing. This should be 2X ell_max in the measurements.

use_binned_l=False  #FIXME: to speed up computation if using pseudo-cl inside mcmc. Needs to be tested. Leave it false for now.

SSV_cov=False # we donot have good model for super sample and tri-spectrum. We can chat about implementing some approximate analytical forms.
tidal_SSV_cov=False
Tri_cov=False 

bin_xi=True
theta_bins=np.logspace(np.log10(1./60),1,20)

In [8]:
def SkyLens_cls(nside, l,  
                dndz_dict, gal_window_dict, 
                cmb_SN_file, cmb_window_map_arr,
                z_cmb = 1090, zmax_cmb = 1090,
                bg1 = None, bz1 = None, use_window = True,
                zmin = 0.0,  mag_fact = 0, cmb_SN_file = None, 
                gal_mask = None,  = None, 
                zmax_gal = 1.7, nz = 75, Win = None):
    """Returns Skylens Cls based on given maps.
    
    Inputs:
        nside (int) : nside for healpy
        l (array) : multipoles to evaluate 
        z_cmb (float) : redshift of CMB 
        dndz_dict (dict) : dictionary containing dndz file location per tomographic bin
        gal_window_dict (dict) : dictionary containing galaxy window function file 
                                location per tomographic bin
        cmb_SN_file (str) : CMB noise curve file location 
        cmb_window_map_arr (str) : CMB window function file location
        use_window (bool) : whether to evaluate window function
        bg1 (float) : linear bias term for galaxies
        bz1 (dict) : redshift dependent galaxy bias
        zmin (float) : min redshift for cross-correlation
        zmax_cmb (float) : maximum redshift where CMB lensing kernel should be integrated up to
        mag_fact (float) : magnification bias 
        gal_mask (str) : file location for galaxy mask 
        zmax_gal (float) : max redshift for galaxy sample
        nz (int) : number of redshifts where P(k) will be evaluated
        Win (dict) : optional dict; pass saved window calculated before
        
    Returns:
        results (dict) : Skylens dict containing Cls, pCls, coupling matrices
    """
    
    results = {}
            
    #tomographed redshift bins for the galaxies
    zl_bin = utils.DESI_elg_bins(ntomo_bins = len(dndz_dict), nside = NSIDE, bg1 = bg1, bz1 = bz1, l=l, 
                         dndz_arr = dndz_dict, gal_mask = gal_mask,
                         gal_window_arr = gal_window_dict)

    np.array([cmb_window_map_file])
    #redshift bins for cmb
    zs_bin = utils.cmb_bins_here(zs = z_cmb, l=l, nside = NSIDE, zmax_cmb = zmax_cmb, 
                                 SN_file = cmb_SN_file, 
                          cmb_window_map_arr = cmb_window_map_arr) # lensing source bin
    
    #names of maps
    corr_kk=('kappa','kappa')
    corr_gg=('galaxy','galaxy')
    corr_gk=('galaxy','kappa')
    corrs=[corr_kk, corr_gg, corr_gk]
    
    tmpz1 = np.linspace(max(zmin, 1e-4), zmax_gal, nz)
    tmpz2 = np.logspace(-4, np.log10(zmax), nz) #
    z_PS = np.sort(np.unique(np.around(np.append(tmpz1, tmpz2), decimals = 3))) #redshifts where P(k) will be evaluated
    print("z_PS: ", len(z_PS))
    if Win is not None:
        kappa0 = skylens.Skylens(kappa_zbins=zs_bin,do_cov=do_cov,bin_cl=bin_cl,l_bins=l_bins,l=l0, galaxy_zbins=zl_bin,
                                       use_window=use_window,Tri_cov=Tri_cov, Win = Win,
                                       use_binned_l=use_binned_l,wigner_files=wigner_files,
                                       SSV_cov=SSV_cov,tidal_SSV_cov=tidal_SSV_cov,
                                       store_win=store_win,window_lmax=window_lmax,
                                       corrs=corrs, scheduler_info=client.scheduler_info(), log_z_PS=1,
                                       cosmo_params = cosmo_params, z_PS=z_PS, pk_params = pk_params)
        
    else:
        kappa0 = skylens.Skylens(kappa_zbins=zs_bin,do_cov=do_cov,bin_cl=bin_cl,l_bins=l_bins,l=l0, galaxy_zbins=zl_bin,
                                       use_window=use_window,Tri_cov=Tri_cov, #Win = Win,
                                       use_binned_l=use_binned_l,wigner_files=wigner_files,
                                       SSV_cov=SSV_cov,tidal_SSV_cov=tidal_SSV_cov,
                                       store_win=store_win,window_lmax=window_lmax,
                                       corrs=corrs, scheduler_info=client.scheduler_info(), log_z_PS=1,
                                       cosmo_params = cosmo_params, z_PS=z_PS, pk_params = pk_params)
    results['kappa0'] = kappa0
    
    return results

In [9]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://128.55.224.43:36925/status,
Dashboard: http://128.55.224.43:36925/status,Workers: 1
Total threads: 4,Total memory: 139.70 GiB
Status: running,Using processes: False
Comm: inproc://128.55.224.43/22625/1,Workers: 1
Dashboard: http://128.55.224.43:36925/status,Total threads: 4
Started: 28 minutes ago,Total memory: 139.70 GiB
Comm: inproc://128.55.224.43/22625/4,Total threads: 4
Dashboard: http://128.55.224.43:40135/status,Memory: 139.70 GiB
Nanny: None,


In [11]:
"""Relevant files for tomography"""
wtype = 'nnp' #window type 
cmb_SN_file = '/global/cscratch1/sd/tanveerk/cmb/lensing/lensing/MV/nlkk.dat'
gal_mask = '../../imaging-sys-covariance/dat/mask_bool_dr9.npy'
cmb_window_map_file = '/global/cscratch1/sd/tanveerk/cmb/lensing/mask_rotated_eq_nside_1024.npy'
dndz_file = '/global/homes/t/tanveerk/desi-lensing-cc/dat/nz_fuji_singletomo_number.csv'
gal_window_map_file = "/global/homes/t/tanveerk/imaging-sys-covariance/dat/Wg_map_nnp.npy"

dndz_dict = np.array([dndz_file])
gal_window_dict = np.array([gal_window_map_file])

In [12]:
pk_params={'non_linear':1,'kmax':30,'kmin':3.e-4,'nk':500, 'scenario':'dmo', 
                 'halofit_version':'takahashi', 'pk_func' :'camb_pk'}

In [ ]:
#define cosmology object
zrange = np.arange(0.05, 1.6, 0.1)
tmpcosmo = skylens.cosmology(cosmo_params=cosmo_params)